In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
from IPython.display import IFrame

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from generator import RoadNetwork
import osmnx as ox
import pandas as pd
import geopandas as gpd
import numpy as np
# import fmm
from shapely.geometry import LineString
import swifter

In [ ]:
network = RoadNetwork("Porto, Portugal", network_type="drive", retain_all=True, truncate_by_edge=True)

In [ ]:
import folium
gdf_nodes, gdf_edges = ox.graph_to_gdfs(network.G)
bounds = gdf_nodes.geometry.total_bounds
print(bounds)
m = folium.Map(
    location=[41.1372482, -8.689151],
    zoom_start=11,
)

folium.Rectangle(bounds=[(bounds[1], bounds[0]),(bounds[3], bounds[2])], color='#ff7800', fill=True, fill_color='#ffff00', fill_opacity=0.2).add_to(m)
m

In [ ]:
from shapely.geometry import box

bbox = box(*bounds)
poly_gdf = gpd.GeoDataFrame([1], geometry=[bbox], crs="EPSG:4326")
poly_gdf

In [ ]:
from shapely import wkt
df = pd.read_csv("../datasets/trajectories/Porto/mapped.csv")
df['POLYLINE'] = df['POLYLINE'].swifter.apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326', geometry="POLYLINE")

In [ ]:
gdf_clipped = gdf.clip(poly_gdf, keep_geom_type=True)

In [ ]:
network.save(path="../osm_data/porto")


In [ ]:
G = network.G
gdf_nodes, gdf_edges = ox.graph_to_gdfs(
        G,
        nodes=True, edges=True,
        node_geometry=True,
        fill_edge_geometry=True)

gdf_edges.describe(include='all')

In [ ]:
m1 = ox.plot_graph_folium(G, popup_attribute="name", weight=2, color="#FFFFFF", tiles="cartodbdark_matter")
filepath = "graph.html"
m1.save(filepath)
#IFrame(filepath, width=600, height=500)

In [ ]:
# load porto trajectory dataset
td = pd.read_csv("../datasets/trajectories/Porto/train.csv")

In [ ]:
td = td[td["MISSING_DATA"]==False]
sample = np.matrix(td.loc[0, "POLYLINE"]).reshape(-1, 2)
gtd = gpd.GeoDataFrame(data=sample, index=np.arange(1, len(sample)+1), columns=["LONG", "LAT"], crs="EPSG:4326", geometry=gpd.points_from_xy(sample[:, 0], sample[:, 1]))
print(gtd.to_wkt())
#network.map_trajectorie(coordinates=gtd)

In [ ]:
df = td = pd.read_csv("../datasets/trajectories/Porto/road-segment-mapping.csv", sep=";")

In [ ]:
#df.dropna(inplace=True)
df.head()

In [ ]:

x = 5e-5

x